In [1]:
from datasets import load_dataset, load_from_disk
import wordninja
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"
import re
import glob
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import datasets
import transformers
from transformers import AutoTokenizer, default_data_collator, get_scheduler
from hf_transformers.src.transformers.models.bert.configuration_bert import BertConfig
from hf_transformers.src.transformers.models.bert.modeling_bert import BertForPreTraining

import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader

import sys
import math

import collections


from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)


/usr/local/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import transformers
transformers.__version__

'4.31.0.dev0'

In [3]:
#NOTE
#ここで記号を区切ってwordninjaで塊除去を行っている
#なお, かなり時間がかかるので実行には注意が必要
#もう少し賢く書きたかったが, その書き方を思いつく時間とナイーブに実行する時間はどうせ同じぐらいだろう
def preprocess_log(text):
    text = text.replace('[', " ")
    text = text.replace(".", " ")
    text = text.replace(",", " ")
    text = text.replace(":", " ")
    text = text.replace("/", " ")
    text = text.replace(";", " ")
    text = text.replace("=", " ")
    text = text.replace("*", " ")
    text = text.replace("_", " ")
    text = text.replace("-", " ")
    text = text.lower()
    text = " ".join(wordninja.split(text))
    remove_num = lambda eg : " ".join([word for word in eg.split(" ") if not word.isdigit()])
    return remove_num(text)

def preprocess_log_batch(example):
    return {
        "input": [preprocess_log(l) for l in example["input"]],
        "summary": [preprocess_log(l) for l in example["summary"]]
    }

In [4]:
def log2list(path):
    with open(path, "r") as f:
        lines = [line.rstrip("\n") for line in f]
    return lines

#ログデータのロード
hdfs = log2list("./LogSummary/data/summary/logs/hdfs.txt")
bgl = log2list("./LogSummary/data/summary/logs/bgl.txt")
hpc = log2list("./LogSummary/data/summary/logs/HPC.txt")
zookeeper = log2list("./LogSummary/data/summary/logs/Zookeeper.txt")
proxifier = log2list("./LogSummary/data/summary/logs/Proxifier.txt")
spark = log2list("./LogSummary/data/summary/logs/spark.txt")

In [5]:
summary_data ={
    "hdfs":hdfs,
    "bgl":bgl,
    "hpc":hpc,
    "zookeeper":zookeeper, 
    "proxifier":proxifier,
    "spark":spark
}

regexp = re.compile(r"#([0-9]+)#")

text_summary_pairs = {}
summary_dict = {}
curr_key = "default"
summary_comes_next = False

for data_name in summary_data.keys():
    text_summary_pairs[data_name] = {}
    for l in summary_data[data_name]:
        if not l:
            pass
        elif regexp.search(l):
            text_summary_pairs[data_name][l] = []
            curr_key=l
        elif "#summary:#" in l:
            summary_comes_next = True
        elif summary_comes_next:
            for elem in text_summary_pairs[data_name][curr_key]:
                elem.append(l)
            summary_comes_next = False
        else:
            text_summary_pairs[data_name][curr_key].append([l])

In [6]:
input_output_pairs = []
for key in text_summary_pairs.keys():
    for k in text_summary_pairs[key]:
        pair = text_summary_pairs[key][k]
        input_output_pairs += pair

invalid_indice = [i for i, pair in enumerate(input_output_pairs) if len(pair)!=2]
for i in invalid_indice:
    input_output_pairs.pop(i)


In [36]:
df_summary.shape

(11980, 2)

In [7]:
df_summary = pd.DataFrame(
    {
        "input": np.array([p[0] for p in input_output_pairs]), 
        "summary": np.array([p[1] for p in input_output_pairs])
    }
)

In [8]:
df_summary.to_csv("./logdata/log_summary_pairs.csv")

In [9]:
dataset_summary = load_dataset("csv", data_files="./logdata/log_summary_pairs.csv")
dataset_summary = dataset_summary["train"].train_test_split(0.2)
dataset_summary = dataset_summary.remove_columns(['Unnamed: 0'])
dataset_summary

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1143.80it/s]


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-00dcddb0e73b1160/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 649.27it/s]


DatasetDict({
    train: Dataset({
        features: ['input', 'summary'],
        num_rows: 9584
    })
    test: Dataset({
        features: ['input', 'summary'],
        num_rows: 2396
    })
})

In [10]:
train_data = dataset_summary["train"]
test_valid = dataset_summary["test"].train_test_split(0.5)
test_dataset = test_valid["test"]
eval_dataset = test_valid["train"]
test_valid

DatasetDict({
    train: Dataset({
        features: ['input', 'summary'],
        num_rows: 1198
    })
    test: Dataset({
        features: ['input', 'summary'],
        num_rows: 1198
    })
})

In [11]:
train_data = train_data.map(
    preprocess_log_batch,
    batched=True,
)

In [12]:
eval_dataset = eval_dataset.map(
    preprocess_log_batch,
    batched=True,
)

test_dataset = test_dataset.map(
    preprocess_log_batch,
    batched=True,
)

In [13]:
log_tokenizer = AutoTokenizer.from_pretrained("./tokenizers/log_tokenizer_from_old_large/")
log_tokenizer_w_n = AutoTokenizer.from_pretrained("./tokenizers/log_tokenizer_from_old_without_numbers/")
batch_size=16  # change to 16 for full training
encoder_max_length=180
decoder_max_length=180

def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    inputs = log_tokenizer_w_n(batch["input"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = log_tokenizer_w_n(batch["summary"], padding="max_length", truncation=True, max_length=decoder_max_length)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["decoder_attention_mask"] = outputs.attention_mask
    batch["labels"] = outputs.input_ids.copy()

    # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`. 
    # We have to make sure that the PAD token is ignored
    batch["labels"] = [[-100 if token == log_tokenizer_w_n.pad_token_id else token for token in labels] for labels in batch["labels"]]

    return batch


In [14]:
train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["input", "summary"]
)
train_data

Dataset({
    features: ['input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask', 'labels'],
    num_rows: 9584
})

In [15]:
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

In [16]:
eval_dataset = eval_dataset.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["input", "summary"]
)

eval_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

In [17]:
MODEL_MAX_LENGTH = 180
log_tokenizer_w_n = AutoTokenizer.from_pretrained("./tokenizers/log_tokenizer_from_old_without_numbers/", model_max_length=MODEL_MAX_LENGTH, padding=True)
#Unilog元論文の実験に準拠
#https://arxiv.org/pdf/2112.03159.pdf
unilogConfig= BertConfig(
    is_unilog=True,
    attention_probs_dropout_prob=0.3,
    hidden_dropout_prob=0.3,
    num_attention_heads=4,
    hidden_size=128,
    intermediate_size=512,
    vocab_size=log_tokenizer_w_n.vocab_size,
    num_hidden_layers=3
)
unilogConfig

BertConfig {
  "attention_probs_dropout_prob": 0.3,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.3,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "is_only_mlm": false,
  "is_unilog": true,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 4,
  "num_hidden_layers": 3,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.31.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 4075
}

In [18]:
"""
とりあえずUnlogモデルをBERTの派生じゃなくて, 一から作る必要がある


"""

'\nとりあえずUnlogモデルをBERTの派生じゃなくて, 一から作る必要がある\n\n\n'

In [19]:
from hf_transformers.src.transformers.models.encoder_decoder import EncoderDecoderConfig, EncoderDecoderModel
config_encoder = unilogConfig
config_decoder = BertConfig(
    is_unilog=True,
    attention_probs_dropout_prob=0.3,
    hidden_dropout_prob=0.3,
    num_attention_heads=4,
    hidden_size=128,
    intermediate_size=512,
    vocab_size=log_tokenizer_w_n.vocab_size,
    num_hidden_layers=3,
    is_decoder=True,
    # add_cross_attention=True
)

In [42]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "./logdata/unilog_pretrain_preln_on_attentions_0/"
unilog2unilog = EncoderDecoderModel.from_encoder_decoder_pretrained(model_path, model_path, encoder_config=unilogConfig, decoder_config=unilogConfig)
unilog2unilog.to(device)

Some weights of BertModel were not initialized from the model checkpoint at ./logdata/unilog_pretrain_preln_on_attentions_0/ and are newly initialized: ['bert.encoder.layer.2.crossattention.self.value.bias', 'bert.encoder.layer.2.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.1.crossattention.self.query.bias', 'bert.encoder.layer.2.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.2.crossattention.LayerNorm.bias', 'bert.encoder.layer.1.crossattention.output.logact.linear.bias', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.2.crossattention.self.key.weight', 'bert.encoder.layer.1.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.LayerNorm.weight', 'bert.encoder.layer.2.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.output.log

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4075, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.3, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.3, inplace=False)
            )
            (output): UnilogBertSelfOutput(
              (logact): LogACT(
                (linear): Line

In [21]:
# unilog2unilog.config.to_json_file("./logdata/unilog_summary/checkpoint-64/config.json")

In [22]:
#WARNING
# !pip install -e .

In [43]:
# set special tokens
# BERTのトークナイザのCLSとSEPをそれぞれBOSとEOSに
unilog2unilog.config.decoder_start_token_id = log_tokenizer_w_n.cls_token_id
unilog2unilog.config.eos_token_id = log_tokenizer_w_n.sep_token_id
unilog2unilog.config.pad_token_id = log_tokenizer_w_n.pad_token_id

# sensible parameters for beam search
unilog2unilog.config.vocab_size = unilog2unilog.config.decoder.vocab_size
unilog2unilog.config.max_length = 181
unilog2unilog.config.min_length = 5
unilog2unilog.config.no_repeat_ngram_size = 2
unilog2unilog.config.early_stopping = True
unilog2unilog.config.length_penalty = 2.0
unilog2unilog.config.num_beams = 4

In [44]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = log_tokenizer_w_n.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = log_tokenizer_w_n.pad_token_id
    label_str = log_tokenizer_w_n.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1"])["rouge1"].mid

    return {
        "rouge1_precision": round(rouge_output.precision, 4),
        "rouge1_recall": round(rouge_output.recall, 4),
        "rouge1_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [45]:
# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
    output_dir="./logdata/unilog_summary_exp03",
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    logging_steps=2,  # set to 1000 for full training
    save_steps=16,  # set to 500 for full training
    eval_steps=4,  # set to 8000 for full training
    warmup_steps=1,  # set to 2000 for full training
    max_steps=64, # delete for full training
    overwrite_output_dir=True,
    save_total_limit=3,
    fp16=True,
)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=unilog2unilog,
    tokenizer=log_tokenizer_w_n,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=eval_dataset,
)
trainer.train()

/home/hf_transformers/src/transformers/optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/home/hf_transformers/src/transformers/models/encoder_decoder/modeling_encoder_decoder.py:654: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Step,Training Loss,Validation Loss,Rouge1 Precision,Rouge1 Recall,Rouge1 Fmeasure
4,8.109700,7.962016,0.013000,0.300600,0.024700
8,7.420300,7.664610,0.013200,0.301400,0.024900
12,7.619200,7.506829,0.013600,0.312300,0.025700
16,7.354100,7.449481,0.007700,0.169000,0.014600
20,7.455700,7.405128,0.008600,0.184000,0.016200
24,7.431900,7.365011,0.010900,0.239000,0.020600
28,7.300300,7.331917,0.011000,0.240300,0.020700
32,7.270000,7.300555,0.011400,0.248700,0.021500
36,7.131200,7.269632,0.013800,0.312600,0.026100
40,6.994700,7.243221,0.014900,0.337100,0.028200


/home/hf_transformers/src/transformers/generation/utils.py:1260: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  "You have modified the pretrained model configuration to control generation. This is a"
/home/hf_transformers/src/transformers/models/encoder_decoder/modeling_encoder_decoder.py:654: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/home/h

TrainOutput(global_step=64, training_loss=7.3194515109062195, metrics={'train_runtime': 3925.9268, 'train_samples_per_second': 0.261, 'train_steps_per_second': 0.016, 'total_flos': 2479077826560.0, 'train_loss': 7.3194515109062195, 'epoch': 0.11})

In [53]:
test_dataseta

Dataset({
    features: ['input', 'summary'],
    num_rows: 1198
})

In [31]:
model = EncoderDecoderModel.from_pretrained("./logdata/unilog_summary/checkpoint-64/")
model.to(device)

# only use 16 training examples for notebook - DELETE LINE FOR FULL TRAINING
batch_size = 16  # change to 64 for full evaluation

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    # cut off at BERT max length 512
    inputs = log_tokenizer_w_n(batch["input"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    outputs = model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = log_tokenizer_w_n.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

In [38]:
results = test_dataset.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["input"])

pred_str = results["pred"]
label_str = results["summary"]

rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1"])["rouge1"].mid

print(rouge_output)

Map:   0%|          | 0/1198 [00:00<?, ? examples/s]/home/hf_transformers/src/transformers/generation/utils.py:1357: UserWarning: Using `max_length`'s default (180) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


Score(precision=0.011634847651113205, recall=0.265060221620865, fmeasure=0.022053094350537643)


In [33]:
results_with_input = test_dataset.map(generate_summary, batched=True, batch_size=batch_size)


In [34]:
df_result = results_with_input.to_pandas()
print(df_result["input"][50])
print("Sum:", df_result["summary"][50])

ras kernel fatal data t lb error interrupt
Sum: error interrupt


wandb: Network error (ReadTimeout), entering retry loop.


In [35]:
df_result

,input,summary,pred
0,info df s fs name system block name system add...,receiving block packet responder terminating r...,block block block info block info info block b...
1,info df s data node packet responder packet re...,receiving block packet responder terminating r...,block block block info block info info block b...
2,switch module fan fan speeds,fan speeds,block block block info block info info block b...
3,play google com open through proxy proxy c se ...,open through proxy bytes sent bytes received c...,block block block info block info info block b...
4,proxy c se cu hk edu hk open through proxy pro...,open through proxy bytes sent bytes received c...,block block block info block info info block b...
...,...,...,...
1193,switch module fan fan speeds,fan speeds,block block block info block info info block b...
1194,info executor executor finished task in stage ...,running task finished task partition not found...,block block block info block info info block b...
1195,ras kernel info instruction cache parity error...,instruction cache parity error corrected,block block block info block info info block b...
1196,proxy c se cu hk edu hk open through proxy pro...,open through proxy bytes sent bytes received c...,block block block info block info info block b...


In [46]:
df_summary["input"][0]

0        INFO dfs.DataNode$DataXceiver : Receiving bloc...
1        INFO dfs.DataNode$DataXceiver : Receiving bloc...
2        INFO dfs.DataNode$DataXceiver : Receiving bloc...
3        INFO dfs.DataNode$DataXceiver : Receiving bloc...
4        INFO dfs.DataNode$PacketResponder : PacketResp...
                               ...                        
11975       INFO storage.BlockManager : Removing RDD 12744
11976       INFO storage.BlockManager : Removing RDD 12740
11977       INFO storage.BlockManager : Removing RDD 12736
11978       INFO storage.BlockManager : Removing RDD 12732
11979       INFO storage.BlockManager : Removing RDD 12728
Name: input, Length: 11980, dtype: object